In [ ]:
# LINE, SKU, DESCRIPTION LINE ITEM COMMENTS, UNIT COST/ RETAIL PRICE, QTY, ITEM TOTAL

In [5]:
import pdfplumber
import pandas as pd
import re

In [229]:
file_path = "static/old.pdf"
with pdfplumber.open(file_path) as pdf:
    # table = pdf.pages[0]
    table = pdf.pages[0].extract_tables()
    table_n = pdf.pages[0].find_tables()
    # table = pdf.pages[0].extract_table(table_settings={
    #                                                     "vertical_strategy": "text",
    #                                                     "horizontal_strategy": "explicit",
    #                                                     "explicit_horizontal_lines": [50,50,50,50],
    #                                                     "text_x_tolerance": 10,
    #                                                    })
# table

In [230]:
table_n[0].bbox

(301.00136939827576, 54.88702310904057, 565.6243149003191, 150.93164605118113)

In [214]:
table.sort(key=len)
table = table[-2:]

## Line Item Part

In [219]:
data_frame = pd.DataFrame(table[1])
data_frame.columns = data_frame.iloc[0]
data_frame = data_frame.drop(0)

In [220]:
data_frame

,LINE,SKU,VENDOR\nPN,UPC/GTIN,DESCRIPTION LINE ITEM\nCOMMENTS,MARKS AND\nNUMBERS,UNIT COST/\nRETAIL\nPRICE,QTY,UOM,ITEM\nTOTAL
1,1,1494819,,,Item Detail: INNO FOODS\nALMOND FRUIT CRUNCH\n...,,Unit Price: 13.19,6720.0,Each,"88,636.80"
2,2,1552820,,,Item Detail: INNO ORGANIC\nGRAIN FREE KETO GRA...,,Unit Price:\n244.12,576.0,Case,"140,613.12"
3,2,# of Line\nItems,Merchandise Total,None,None,None,None,7296,,"229,249.92"


In [221]:
data_frame.columns = data_frame.columns.str.replace('\n', ' ')
required_columns = ['LINE', 'SKU', 'DESCRIPTION LINE ITEM COMMENTS', 'UNIT COST/ RETAIL PRICE', 'QTY', 'ITEM TOTAL']
required_columns = list(map(lambda text: re.sub(r'\s+', ' ', text.strip()), required_columns))
selected_df = data_frame[required_columns].copy()

selected_df = selected_df.iloc[:-1]
selected_df = selected_df.replace('\n',' ', regex=True)
final_df = selected_df.set_index("LINE")

In [222]:
final_df

,SKU,DESCRIPTION LINE ITEM COMMENTS,UNIT COST/ RETAIL PRICE,QTY,ITEM TOTAL
LINE,,,,,
1,1494819,Item Detail: INNO FOODS ALMOND FRUIT CRUNCH 90...,Unit Price: 13.19,6720.0,"88,636.80"
2,1552820,Item Detail: INNO ORGANIC GRAIN FREE KETO GRAN...,Unit Price: 244.12,576.0,"140,613.12"


In [223]:
json_output = final_df.to_json(orient = 'columns')
json_output

'{"SKU":{"1":"1494819","2":"1552820"},"DESCRIPTION LINE ITEM COMMENTS":{"1":"Item Detail: INNO FOODS ALMOND FRUIT CRUNCH 900G P280","2":"Item Detail: INNO ORGANIC GRAIN FREE KETO GRANOLA 600G P272"},"UNIT COST\\/ RETAIL PRICE":{"1":"Unit Price: 13.19","2":"Unit Price: 244.12"},"QTY":{"1":"6720.0","2":"576.0"},"ITEM TOTAL":{"1":"88,636.80","2":"140,613.12"}}'

In [99]:
selected_df.to_csv("output.csv")

## Other Parts 

In [266]:
from itertools import chain
import re

In [264]:
item = []
result = list(chain(table[0][0], table[0][1], table[0][3]))
result

['PO Date:\n08/16/2022',
 'Requested Delivery Date:\n08/25/2022',
 'Requested Ship Date:\n08/24/2022',
 'Cancel Date:\n08/29/2022',
 'Vendor #:\n0007764600',
 'Department #:\n13COSTCO DEPARTMENT']

In [268]:
pattern = r"\n(.+)"
match = re.search(pattern, result[0])

In [272]:
match.group(1)

'08/16/2022'

In [263]:
df1 = {'Short_descp':Short_descp,'TYPE':Type,'SIZE':size,'LENGTH':length, 'MATERIAL':material}

['PO Date:\n08/16/2022', 'Requested Delivery Date:\n08/25/2022']

In [249]:
list_other = ["PO Date", "Requested Ship Date", "Requested Delivery Date", "Cancel Date", "Vendor #"]

In [ ]:
for items in list_other:
    if items in result:
        print("g")

In [248]:
dic_other = {}
for i in data_frame.items():
    print(i[1])

0                PO Date:\n08/16/2022
1    Requested Ship Date:\n08/24/2022
2                    Delivery Window:
3               Vendor #:\n0007764600
Name: 0, dtype: object
0    Requested Delivery Date:\n08/25/2022
1                Cancel Date:\n08/29/2022
2                        Shipping Window:
3      Department #:\n13COSTCO DEPARTMENT
Name: 1, dtype: object
